In [1]:
import torch.nn as nn
import torch.nn.functional as F
import net_size_utils as nsu

class Net(nn.Module):
    def __init__(self, kernel_size=5, channel_sizes=None):
        super().__init__()
        if channel_sizes == None:
            channel_sizes = [6, 16]
        self.channel_sizes = channel_sizes
        self.kernel_size = kernel_size
        self.conv1 = nn.Conv2d(3, channel_sizes[0], kernel_size)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(channel_sizes[0], channel_sizes[1], kernel_size)
        conv_height, conv_width = self.conv_out_size(224, 224)
        linear_input_size = channel_sizes[1] * conv_width * conv_height
        self.fc1 = nn.Linear(linear_input_size, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def conv_out_size(self, height, width):
        dim = (height, width)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        return dim

In [2]:
class Net3Conv(nn.Module):
    def __init__(self, kernel_size=5, channel_sizes=None):
        super().__init__()
        if channel_sizes == None:
            channel_sizes = [8, 16, 32]
        self.channel_sizes = channel_sizes
        self.kernel_size = kernel_size
        self.conv1 = nn.Conv2d(3, channel_sizes[0], kernel_size)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(channel_sizes[0], channel_sizes[1], kernel_size)
        self.conv3 = nn.Conv2d(channel_sizes[1], channel_sizes[2], kernel_size)
        conv_height, conv_width = self.conv_out_size(224, 224)
        linear_input_size = channel_sizes[2] * conv_width * conv_height
        self.fc1 = nn.Linear(linear_input_size, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def conv_out_size(self, height, width):
        dim = (height, width)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        return dim

In [3]:
import statistics
import torch.utils.data as td
import net_training
from birds_dataset import Birds270Dataset
import torch
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import pandas as pd

# Splits a dataset randomly into a train and test set. The size of test set is 80% of the whole dataset.
# Then it trains the network
def train_net_random_dataset_split(net, dataset, epochs, optimizer, batch_size):
    train_set_size = int(len(dataset)*0.8)
    test_set_size = len(dataset)-train_set_size
    train_dataset, test_dataset = td.random_split(dataset, [train_set_size, test_set_size])
    label_set = dataset.get_label_set()
    train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size, shuffle=True)
    results = net_training.train_and_evaluate(net, train_dataloader, test_dataloader, label_set,
                                 epochs=epochs, optimizer=optimizer, print_results=True)
    return results

# Creates a few networks and trains them using a random split of the dataset.
# The number of created networks is in the "repeat" argument
# It returns the final validation results for each network
def cross_validate_net(net_generator, dataset, repeat=5, epochs=20, batch_size=32):
    all_results = []
    for i in range(repeat):
        print(f"Training network {i+1} ...")
        net = net_generator()
        results = train_net_random_dataset_split(net, dataset, epochs, optimizer=None, batch_size=batch_size)
        all_results.append(results)
        net_training.print_final_results(results)
    return all_results
    
    
def print_validation_results(cross_validation_results):
    losses = [r["loss"] for r in cross_validation_results]
    accuracies = [r["accuracy"] for r in cross_validation_results]
    print("Losses: ", losses)
    print("Loss:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(losses), statistics.stdev(losses)))
    print("Accuracies: ", accuracies)
    print("Accuracy:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(accuracies), statistics.stdev(accuracies)))
    
def results_to_dataframe(cross_validation_results):
    normalized = pd.json_normalize(cross_validation_results)
    return normalized
    


In [4]:
dataset_dir = "../data/birds270"
selected_birds = ["ALBATROSS", "BALD EAGLE", "BARN OWL", "EURASIAN MAGPIE", "FLAMINGO",
                  "MALLARD DUCK", "OSTRICH", "PEACOCK", "PELICAN", "TRUMPTER SWAN"]

transform = transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5)) # normalizes colors to range [-1,1]
dataset = Birds270Dataset(dataset_dir,  selected_birds=selected_birds, transform=transform)

def net_generator():
    return Net3Conv(channel_sizes=[16,16,16])

results = cross_validate_net(net_generator, dataset, repeat=5, epochs=100, batch_size=64)
print_validation_results(results)

Training network 1 ...
Epoch 0:
	train loss: 1.984622205919099
	validation loss: 1.4812408620364046, validation accuracy: 49.65753424657534%
	Elapsed time: 0:00:14.375184
Epoch 1:
	train loss: 1.3495200118928585
	validation loss: 1.32729952057747, validation accuracy: 54.109589041095894%
	Elapsed time: 0:00:14.538130
Epoch 2:
	train loss: 1.042765413508407
	validation loss: 1.1655754602118715, validation accuracy: 58.56164383561644%
	Elapsed time: 0:00:14.504096
Epoch 3:
	train loss: 0.8253057534657866
	validation loss: 1.0155425267676785, validation accuracy: 67.8082191780822%
	Elapsed time: 0:00:14.150775
Epoch 4:
	train loss: 0.6750025579491774
	validation loss: 1.02086195227218, validation accuracy: 66.78082191780823%
	Elapsed time: 0:00:14.087721
Epoch 5:
	train loss: 0.4904887873046803
	validation loss: 0.9339355801882809, validation accuracy: 70.54794520547945%
	Elapsed time: 0:00:14.247718
Epoch 6:
	train loss: 0.36189451987502097
	validation loss: 0.9053710797061659, validatio

Epoch 7:
	train loss: 0.20443370110174805
	validation loss: 1.0130576605666173, validation accuracy: 70.2054794520548%
	Elapsed time: 0:00:14.183642
Epoch 8:
	train loss: 0.11718952580711342
	validation loss: 1.255732334639928, validation accuracy: 72.6027397260274%
	Elapsed time: 0:00:14.473314
Epoch 9:
	train loss: 0.06492450708239043
	validation loss: 1.4663722482446122, validation accuracy: 68.4931506849315%
	Elapsed time: 0:00:14.277472
Epoch 10:
	train loss: 0.04426373320265863
	validation loss: 1.3155859610805773, validation accuracy: 72.6027397260274%
	Elapsed time: 0:00:14.460908
Epoch 11:
	train loss: 0.01635439739775034
	validation loss: 1.4505733888443202, validation accuracy: 72.26027397260275%
	Elapsed time: 0:00:14.451338
Epoch 12:
	train loss: 0.005074065508509429
	validation loss: 1.4752247104906056, validation accuracy: 74.31506849315068%
	Elapsed time: 0:00:15.610789
Epoch 13:
	train loss: 0.0018207937629580318
	validation loss: 1.5688377488149357, validation accurac

Epoch 17:
	train loss: 0.012290157539063927
	validation loss: 1.9793866276741028, validation accuracy: 68.4931506849315%
	Elapsed time: 0:00:14.299412
Epoch 18:
	train loss: 0.006975281761566219
	validation loss: 2.2093823380666238, validation accuracy: 69.17808219178082%
	Elapsed time: 0:00:16.954089
Epoch 19:
	train loss: 0.0027490807700421786
	validation loss: 2.156737935053159, validation accuracy: 67.46575342465754%
	Elapsed time: 0:00:14.549804
Epoch 20:
	train loss: 0.0013153759712124912
	validation loss: 2.169198320336538, validation accuracy: 69.52054794520548%
	Elapsed time: 0:00:15.196468
Epoch 21:
	train loss: 0.0007471215928082816
	validation loss: 2.2124942671762753, validation accuracy: 69.17808219178082%
	Elapsed time: 0:00:14.734385
Epoch 22:
	train loss: 0.000559806124939013
	validation loss: 2.2560518049213982, validation accuracy: 68.15068493150685%
	Elapsed time: 0:00:14.772879
Epoch 23:
	train loss: 0.0004442820341867837
	validation loss: 2.2830330280408466, valid

Epoch 8:
	train loss: 0.18649064763536993
	validation loss: 1.1827293340473959, validation accuracy: 66.43835616438356%
	Elapsed time: 0:00:14.280395
Epoch 9:
	train loss: 0.11740172407741611
	validation loss: 1.4266161967630255, validation accuracy: 69.17808219178082%
	Elapsed time: 0:00:14.327691
Epoch 10:
	train loss: 0.07613070996871177
	validation loss: 1.3725741521952903, validation accuracy: 70.54794520547945%
	Elapsed time: 0:00:14.276152
Epoch 11:
	train loss: 0.08991906615565204
	validation loss: 1.774209881482059, validation accuracy: 66.43835616438356%
	Elapsed time: 0:00:14.331704
Epoch 12:
	train loss: 0.07322129326046638
	validation loss: 1.4581777428927487, validation accuracy: 68.4931506849315%
	Elapsed time: 0:00:14.235014
Epoch 13:
	train loss: 0.0231303890214955
	validation loss: 1.354418251612415, validation accuracy: 72.26027397260275%
	Elapsed time: 0:00:14.283650
Epoch 14:
	train loss: 0.005629479153461971
	validation loss: 1.5269650351511288, validation accurac

In [5]:
dataframe = results_to_dataframe(results)
dataframe.to_csv("../results/net_3conv_2fc_channels_16_16_16.csv")